<a href="https://colab.research.google.com/github/nikolasleeb/INFO523_FinalProject/blob/main/ConsumptionData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
import glob, os

# Attach Data

In [12]:
'''Connects to Google Drive to load in data'''

# Connect to and mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Find the dataset in google drive and assign path to variable called 'Match'
matches = glob.glob('/content/drive/MyDrive/**/ConsumptionData.xlsx', recursive=True)
print("Found paths:", matches)

# Using 'Match' load in the dataset using the path and skip the first 2 rows
path = matches[0]
consumption = pd.read_excel(path, engine='openpyxl', skiprows=2)
consumption.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found paths: ['/content/drive/MyDrive/DataMiningFall25/INFO 523 Final Project/Datasets/FuelConsumption/ConsumptionData.xlsx']


,Date,U.S. Product Supplied of Finished Motor Gasoline (Thousand Barrels),East Coast (PADD 1) Product Supplied of Finished Motor Gasoline (Thousand Barrels),Midwest (PADD 2) Product Supplied of Finished Motor Gasoline (Thousand Barrels),Gulf Coast (PADD 3) Product Supplied of Finished Motor Gasoline (Thousand Barrels),Rocky Mountain (PADD 4) Product Supplied of Finished Motor Gasoline (Thousand Barrels),West Coast (PADD 5) Product Supplied of Finished Motor Gasoline (Thousand Barrels)
0,1945-01-15,40310,NaN,NaN,NaN,NaN,NaN
1,1945-02-15,38690,NaN,NaN,NaN,NaN,NaN
2,1945-03-15,42511,NaN,NaN,NaN,NaN,NaN
3,1945-04-15,45351,NaN,NaN,NaN,NaN,NaN
4,1945-05-15,47515,NaN,NaN,NaN,NaN,NaN


In [13]:
colors = { "consumption": "royalblue" }

# Cleaning and Transforming

In [4]:
''' Verify the columns, datatypes, and number of entries'''
consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967 entries, 0 to 966
Data columns (total 7 columns):
 #   Column                                                                                  Non-Null Count  Dtype         
---  ------                                                                                  --------------  -----         
 0   Date                                                                                    967 non-null    datetime64[ns]
 1   U.S. Product Supplied of Finished Motor Gasoline (Thousand Barrels)                     967 non-null    int64         
 2   East Coast (PADD 1) Product Supplied of Finished Motor Gasoline (Thousand Barrels)      535 non-null    float64       
 3   Midwest (PADD 2) Product Supplied of Finished Motor Gasoline (Thousand Barrels)         535 non-null    float64       
 4   Gulf Coast (PADD 3) Product Supplied of Finished Motor Gasoline (Thousand Barrels)      535 non-null    float64       
 5   Rocky Mountain (PADD 4)

In [5]:
''' Data Cleaning for Consumption Data '''

# Keep only needed columns and rename
consumption = consumption.iloc[:, :2]
consumption.columns = ["Date", "Gasoline_Consumption_Thousand_Barrels"]

# Convert Date column to datetime and filter
consumption["Date"] = pd.to_datetime(consumption["Date"]).dt.to_period("M").dt.to_timestamp()

# Filter to keep data between April 1994 and December 2024
consumption = consumption[
    (consumption["Date"] >= "1994-01-01") &
    (consumption["Date"] <= "2024-12-31")
]

# Convert to Millions of Gallons
consumption["Gasoline_Consumption(Millions of Gallons)"] = (
    consumption["Gasoline_Consumption_Thousand_Barrels"] * 1000 * 42 / 1_000_000
)

# Drop the old column
consumption.drop(columns=["Gasoline_Consumption_Thousand_Barrels"], inplace=True)

# Preview
consumption.head(10)

,Date,Gasoline_Consumption(Millions of Gallons)
588,1994-01-01,9088.044
589,1994-02-01,8555.274
590,1994-03-01,9628.206
591,1994-04-01,9530.136
592,1994-05-01,9953.034
593,1994-06-01,9981.888
594,1994-07-01,10265.598
595,1994-08-01,10382.862
596,1994-09-01,9595.404
597,1994-10-01,9827.328


# Summary Stats

In [6]:
consumption.describe()

,Date,Gasoline_Consumption(Millions of Gallons)
count,372,372.000000
mean,2009-06-16 07:52:15.483870976,11211.649242
min,1994-01-01 00:00:00,7390.866000
25%,2001-09-23 12:00:00,10715.638500
50%,2009-06-16 00:00:00,11334.141000
75%,2017-03-08 18:00:00,11805.402000
max,2024-12-01 00:00:00,12803.448000
std,NaN,820.504759


In [7]:
'''' View skewness and kurtosis of consumption data '''

# Compute skewness and kurtosis for consumption
cons_skew = skew(consumption["Gasoline_Consumption(Millions of Gallons)"], nan_policy='omit')
cons_kurt = kurtosis(consumption["Gasoline_Consumption(Millions of Gallons)"], nan_policy='omit')

# Display results
print(f"Gasoline Consumption — Skewness: {cons_skew:.2f}, Kurtosis: {cons_kurt:.2f}")

Gasoline Consumption — Skewness: -0.84, Kurtosis: 0.96


In [8]:
''' View consumption date range and total months of data '''

print("Earliest date:", consumption["Date"].min()) # Confirming data starts at Apr 1994 to match Prices data
print("Latest date:", consumption["Date"].max()) # Confirming data ends at Dec 2024 to match Prices data
print("Total months of data:", len(consumption))

Earliest date: 1994-01-01 00:00:00
Latest date: 2024-12-01 00:00:00
Total months of data: 372


In [9]:
''' Calculate and print average monthly consumption, standard deviation, and coefficient of variation '''

mean = consumption["Gasoline_Consumption(Millions of Gallons)"].mean()
std = consumption["Gasoline_Consumption(Millions of Gallons)"].std()
cv = (std / mean) * 100     # coefficient of variation (%)
                            # CV means how consistent the data is relative to the mean
                            # 7.5% means the standard deviation is 7.5% of the mean value

print(f"Average monthly consumption: {mean:,.2f} million gallons")
print(f"Standard deviation: {std:,.2f} million gallons")
print(f"Coefficient of variation: {cv:.2f}%")

Average monthly consumption: 11,211.65 million gallons
Standard deviation: 820.50 million gallons
Coefficient of variation: 7.32%


In [10]:
''' Create a histogram to visualize the distribution of monthly gasoline consumption '''

# Create histogram
figc1 = px.histogram(
    consumption,
    x="Gasoline_Consumption(Millions of Gallons)",
    nbins=25,
    title="Distribution of Monthly Gasoline Consumption",
    labels={"Gasoline_Consumption(Millions of Gallons)": "Millions of Gallons"},
    color_discrete_sequence=[colors["consumption"]]
)

# Customize layout
figc1.update_layout(
    template="plotly_white",
    title_x=0.5,
    xaxis_title="Millions of Gallons",
    yaxis_title="Frequency",
    xaxis=dict(showgrid=True, gridcolor="lightgray"),
    yaxis=dict(showgrid=True, gridcolor="lightgray"),
    hovermode="x unified",
    height=500
)

figc1.update_traces(marker_line_color="black", marker_line_width=1, opacity=1)

figc1.show()

In [11]:
''' Outlier Detection '''

consumption["Z_Consumption"] = (
    (consumption["Gasoline_Consumption(Millions of Gallons)"] - consumption["Gasoline_Consumption(Millions of Gallons)"].mean())
    / consumption["Gasoline_Consumption(Millions of Gallons)"].std()
)

# Flag potential outliers (|Z| > 3)
consumption_outliers = consumption[np.abs(consumption["Z_Consumption"]) > 3]

print(f"Detected {len(consumption_outliers)} potential outliers in gasoline consumption.")
display(consumption_outliers[["Date", "Gasoline_Consumption(Millions of Gallons)"]])

# visual outlier detection with box plot
figc2 = px.box(
    consumption,
    y="Gasoline_Consumption(Millions of Gallons)",
    title="Outlier Detection: U.S. Monthly Gasoline Consumption",
    points="all",
)
figc2.update_layout(template="plotly_white", title_x=0.5, height=500)
figc2.show()

Detected 2 potential outliers in gasoline consumption.


,Date,Gasoline_Consumption(Millions of Gallons)
589,1994-02-01,8555.274
903,2020-04-01,7390.866
